In [1]:
import os
os.chdir('..')

from util import hand_to_str
from deck52 import random_deal
import binary
from nn.models import Models
from sample import Sample

from nn.bid_info import BidInfo
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_suit-999000
INFO:tensorflow:Restorin

In [2]:
auction = ['4C']
n_steps = binary.calculate_step_bidding_info(auction, models)
nesw_i = 1
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT85.AKT.K63.K8")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)


In [3]:
p_hcp, p_shp = models.binfo_model.model(A)
p_hcp = p_hcp.reshape((-1, n_steps, 3))[:, -1, :]
p_shp = p_shp.reshape((-1, n_steps, 12))[:, -1, :]

def f_trans_hcp(x): return 4 * x + 10
def f_trans_shp(x): return 1.75 * x + 3.25

p_hcp = f_trans_hcp(p_hcp)
p_shp = f_trans_shp(p_shp)

c_hcp = p_hcp.copy()
c_shp = p_shp.copy()

print(f"c_hcp:{c_hcp}")
print(f"c_shp:{c_shp}")


c_hcp:[[6.24 7.22 8.02]]
c_shp:[[3.2  4.01 3.6  2.18 3.34 3.95 3.72 1.91 1.58 2.11 2.4  7.01]]


In [4]:
auction = ['PAD_START', '2C','4N']
n_steps = binary.calculate_step_bidding_info(auction, models)
n_samples = 10
ns = -1
ew = -1
nesw_i = 3
vuln = [True, False]
hand = binary.parse_hand_f(32)("AJT8653.J.962.92")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)
p_hcp, p_shp = models.binfo_model.model(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[ 4.85 21.18  6.32]]


In [5]:
auction = ['PAD_START', '2C','4N','PASS']
n_steps = binary.calculate_step_bidding_info(auction, models)
n_samples = 10
nesw_i = 0
vuln = [True, False]
hand = binary.parse_hand_f(32)("K4.T9876532.54.6")
A = binary.get_auction_binary(n_steps, auction, nesw_i, hand, vuln, models)
p_hcp, p_shp = models.binfo_model.model(A)
c_hcp = (lambda x: 4 * x + 10)(p_hcp.copy())
print(c_hcp)


[[22.29  7.97  5.1 ]
 [22.92  9.21  4.15]]
